In [1]:
from modular.model import CustomResNet01
from modular.transforms import CustomResnetTransforms
from modular.dataloader import Cifar10SearchDataset
import modular.utils as utils

from torchsummary import summary
import yaml
from yaml.loader import SafeLoader

/Users/bala/exit/envs/torchEnv/lib/python3.10/site-packages/torch_lr_finder/lr_finder.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
with open("modular/params.yaml") as f:
    params = yaml.load(f, Loader=SafeLoader)

In [3]:
means = params["transform_means"]
stds = params["transform_stds"]

train_transforms = CustomResnetTransforms.train_transforms(means=means, stds=stds)
test_transforms = CustomResnetTransforms.test_transforms(means, stds)

In [4]:
train = Cifar10SearchDataset(root='./data', train=True,
                                        download=True, transform=train_transforms)
test = Cifar10SearchDataset(root='./data', train=False,
                                       download=True, transform=test_transforms)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
train_loader = utils.construct_train_loader(train)
test_loader = utils.construct_test_loader(test)

In [6]:
model = CustomResNet01()
summary(model, input_size= (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
            Conv2d-4          [-1, 128, 34, 34]           8,192
       BatchNorm2d-5          [-1, 128, 34, 34]             256
              ReLU-6          [-1, 128, 34, 34]               0
         MaxPool2d-7          [-1, 128, 17, 17]               0
            Conv2d-8          [-1, 128, 17, 17]         147,456
       BatchNorm2d-9          [-1, 128, 17, 17]             256
             ReLU-10          [-1, 128, 17, 17]               0
           Conv2d-11          [-1, 128, 17, 17]         147,456
      BatchNorm2d-12          [-1, 128, 17, 17]             256
             ReLU-13          [-1, 128, 17, 17]               0
           Conv2d-14          [-1, 256,

In [7]:
optimizer = utils.construct_optimizer(model=model, learning_rate=0.03, weight_decay=1e-4)
cross_entropy_loss = utils.construct_cross_entropy_loss()

In [8]:
lr_finder = utils.construct_LR_finder(model=model, optimizer=optimizer, criterion=cross_entropy_loss, device="cpu", 
                          dataloader=train_loader, end_learning_rate=10, 
                          number_of_iterations=200, step_mode="exp")

  0%|          | 0/200 [00:00<?, ?it/s]

RuntimeError: NNPACK SpatialConvolution_updateOutput failed